In [1]:
import time
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

# 特征处理

In [2]:
columns=['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
        '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
        '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
        '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
        '交银装备']

In [3]:
df = pd.read_csv(r'../Data/TRAINSET_STOCK.csv')
df_add1 = pd.read_csv(r'../Data/20190506_STOCK.csv')
df_add2 = pd.read_csv(r'../Data/20190514_STOCK.csv')
df_add3 = pd.read_csv(r'../Data/20190521_STOCK.csv')
df_add4 = pd.read_csv(r'../Data/20190528_STOCK.csv')
df = pd.concat([df,df_add1,df_add2,df_add3,df_add4])
df = df[~df.index.duplicated()]



In [9]:
x = []
y = []
for arr in columns:
    df1=df.loc[df['name']==arr]
    df1=df1[['ts_code','trade_date','name','low','high','close','pct_change','vol','y']]
    df1['vol_change'] = df1['vol'].pct_change()
    df1['low_change'] = df1['low'].pct_change()
    df1['high_change'] = df1['high'].pct_change()
    df1['gaodi']=(df1['high']-df1['low'])/df1['low']
    df1['gaoclo']=(df1['high']-df1['close'])/(df1['high']-df1['low'])
    df1['diclo']=(df1['close']-df1['low'])/(df1['high']-df1['low'])
    df1['pct_change']=df1['pct_change']/10
    df1=df1[['pct_change','vol_change','high_change','low_change','gaodi','gaoclo','diclo','y']]
    df1= df1.dropna()
    dataset=df1.values
    for i in range(len(dataset)-15):
        x.append(dataset[i:(i+10),:8].tolist())
        y.append(dataset[(i+10):(i+15),7,np.newaxis].tolist())
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=30)
xtrain, xtest, ytrain, ytest = np.array(xtrain),np.array(xtest),np.array(ytrain),np.array(ytest)
# xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1]*xtrain.shape[2], 1))
# xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
ytrain = np.reshape(ytrain, (ytrain.shape[0], ytrain.shape[1]))
ytest= np.reshape(ytest, (ytest.shape[0], ytest.shape[1]))


In [10]:
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((30579, 10, 8), (7645, 10, 8), (30579, 5), (7645, 5))

# LSTM

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Activation
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

Using TensorFlow backend.


# 没有交叉验证


In [12]:
regressor = Sequential()
regressor.add(LSTM(units = 70, return_sequences = True, input_shape = (xtrain.shape[1], 8)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 70, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 70,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 70))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 5))
regressor.add(Activation('sigmoid'))
regressor.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
regressor.fit(xtrain, ytrain, epochs = 100, batch_size = 50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
30579/30579 [==============================] - 38s 1ms/step - loss: 4.3487 - acc: 0.1945
Epoch 2/100
30579/30579 [==============================] - 31s 1ms/step - loss: 4.3485 - acc: 0.2596
Epoch 3/100
30579/30579 [==============================] - 32s 1ms/step - loss: 4.3485 - acc: 0.2212
Epoch 4/100
30579/30579 [==============================] - 30s 967us/step - loss: 4.3483 - acc: 0.2655
Epoch 5/100
30579/30579 [==============================] - 32s 1ms/step - loss: 4.3455 - acc: 0.2311
Epoch 6/100
30579/30579 [==============================] - 32s 1ms/step - loss: 4.3352 - acc: 0.2099
Epoch 7/100
30579/30579 [==============================] - 32s 1ms/step - loss: 4.3247 - acc: 0.2366
Epoch 8/100
30579/30579 [==============================] - 31s 1ms/step - loss: 4.3131 - acc: 0.2297
Epoch 9/100
30579/30579 [==============================] - 31s 1ms/step - loss: 4.3010 - acc: 0.2372
Epoch 10/100
30579/30579 [===============

In [14]:
from sklearn.externals import joblib
joblib.dump(regressor,'./lstm.model')


['./lstm.model']

In [15]:
predicted_stock_price = regressor.predict(xtest)
a = []
for i in range(len(ytest)):
    fpr, tpr, thresholds = roc_curve(ytest[i], predicted_stock_price[i], pos_label=1)
    each_a = auc(fpr, tpr)
    a.append(each_a)
a = pd.DataFrame(a)
a = a.dropna()
a.mean()

0    0.921948
dtype: float64

# 预测结果

In [16]:
regressor=joblib.load('./lstm.model')
df_test1 = pd.read_csv(r'../Data/20190521_STOCK.csv')
df_test2 = pd.read_csv(r'../Data/20190528_STOCK.csv')
df_test3 = pd.read_csv(r'../Data/20190604_STOCK.csv')
df_test4 = pd.concat([df_test1,df_test2,df_test3])
# df_test4.iloc[1:3]
# df_test4 = df_test4[~df_test4.index.duplicated()]
df_test4.head()
# df_test4
    


,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y
0,801010,20190506,农林牧渔,3446.17,3324.46,3553.36,3364.21,-208.12,-5.83,251935.0,2814423.0,48.91,3.61,0
1,801020,20190506,采掘,2787.37,2669.82,2790.80,2698.85,-171.23,-5.97,223412.0,1432666.0,13.70,1.21,0
2,801030,20190506,化工,2685.53,2559.36,2687.45,2581.85,-203.82,-7.32,469011.0,4412583.0,18.44,1.84,0
3,801040,20190506,钢铁,2447.10,2323.86,2447.10,2355.31,-138.63,-5.56,142708.0,609866.0,7.54,1.03,0
4,801050,20190506,有色金属,2741.15,2613.66,2758.79,2637.87,-195.66,-6.91,279336.0,2089373.0,31.89,1.90,0


In [17]:
df_test4['vol_change'] = df_test4.groupby(["ts_code"])['vol'].pct_change()
df_test4['low_change'] = df_test4.groupby(["ts_code"])['low'].pct_change()
df_test4['high_change'] = df_test4.groupby(["ts_code"])['high'].pct_change()
df_test4['gaodi']=(df_test4['high']-df_test4['low'])/df_test4['low']
df_test4['gaoclo']=(df_test4['high']-df_test4['close'])/(df_test4['high']-df_test4['low'])
df_test4['diclo']=(df_test4['close']-df_test4['low'])/(df_test4['high']-df_test4['low'])
df_test4['pct_change']=df_test4['pct_change']/10
df_test4=df_test4[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','gaodi','gaoclo','diclo','y']]


# 预测5.29-6.04

In [18]:
# 预测未来五天的结果

colums=[801010, 801020, 801030, 801040, 801050, 801080, 801110, 801120,
        801130, 801140, 801150, 801160, 801170, 801180, 801200, 801210,
        801230, 801250, 801260, 801270, 801280, 801300, 801710, 801720,
        801730, 801740, 801750, 801760, 801770, 801780, 801790, 801880,
        801890, 802600]
range_index = (df_test4.trade_date > 20190521)
df_final = df_test4[range_index]
df_pr=pd.DataFrame()
for arr in colums:
    df3 = df_final.loc[df_final['ts_code']==arr]
    dataset2=df3.values
    x1=dataset2[0:10,3:11].tolist()
    x1=np.array(x1)
    x1=np.reshape(x1, (1,x1.shape[0],x1.shape[1]))
    ytest1=regressor.predict(x1)
    y_p=pd.DataFrame(ytest1)
    y_p=y_p.T
    y_p['ts_code']=arr
    y_p['trade_date']=[20190529,20190530,20190531,20190603,201900604]
    df_pr = pd.concat([df_pr,y_p])
# df_pr=df_me.merge(df_pr,how='left',on='name')
df_pr.columns=['p','ts_code','trade_date']
df_pr=df_pr[['ts_code','trade_date','p']]
range_index = (df_pr.trade_date != 20190607)
df_prl = df_pr[range_index]
df_pr1.to_csv('./结果提交第五周最终.csv')

# 交叉验证

In [20]:
kf = KFold(n_splits=5, shuffle=True, random_state=520)

In [21]:
i=0
df_yanz=pd.DataFrame()
df_jieguo=pd.DataFrame()
for train,test in kf.split(xtrain):
    a = []
    b= []
    X_train, X_valid= xtrain[train,:], xtrain[test,:]
    y_train, y_valid = ytrain[train,:], ytrain[test,:]
    print(i, 'fold')
    regressor1 = Sequential()
    regressor1.add(LSTM(units = 60, return_sequences = True, input_shape = (xtrain.shape[1], 8)))
    regressor1.add(Dropout(0.2))
    regressor1.add(LSTM(units = 60, return_sequences = True))
    regressor1.add(Dropout(0.2))
    regressor1.add(LSTM(units = 60,return_sequences = True))
    regressor1.add(Dropout(0.2))
    regressor1.add(LSTM(units = 60))
    regressor1.add(Dropout(0.2))
    regressor1.add(Dense(units = 5))
    regressor1.add(Activation('sigmoid'))
    regressor1.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    regressor1.fit(X_train,y_train, epochs = 100, batch_size = 80)
    joblib.dump(regressor1,'./lstm{i}.model')
    predicted_stock_price1 = regressor1.predict(X_valid)
    for j in range(len(y_valid)):
        fpr1, tpr1, thresholds1 = roc_curve(y_valid[j],predicted_stock_price1[j], pos_label=1)
        a.append(auc(fpr1, tpr1))
    a=pd.DataFrame(a)
    a=a.dropna()
    print(a.mean())
    predicted_stock_price2 = regressor1.predict(xtest)
    for k in range(len(ytest)):
        fpr2, tpr2, thresholds2 = roc_curve(ytest[k], predicted_stock_price2[k], pos_label=1)
        b.append(auc(fpr2, tpr2))
    b=pd.DataFrame(b)
    b=b.dropna()
    print(b.mean())
    df_yanz=df_yanz+pd.DataFrame(predicted_stock_price2)
    df_pr=pd.DataFrame()
    for arr in colums:
        df4 = df_final.loc[df_final['ts_code']==arr]
        dataset4=df4.values
        x1=dataset2[0:10,3:8].tolist()
        x1=np.array(x1)
        x1=np.reshape(x1, (1,x1.shape[0],x1.shape[1]))
        ytest1=regressor1.predict(x1)
        y_p=pd.DataFrame(ytest1)
        y_p=y_p.T
        y_p['ts_code']=arr
        y_p['trade_date']=[20190529,20190530,20190531,20190603,201900604]
    df_pr = pd.concat([df_pr,y_p])
# df_pr=df_me.merge(df_pr,how='left',on='name')
    df_pr.columns=['i','ts_code','trade_date']
    df_pr=df_pr[['ts_code','trade_date','i']]
    df_jieguo['ts_code']=df_pr['ts_code']
    df_jieguo['trade_date']=df_pr['trade_date']
    df_jieguo['i']=df_pr['i']
    i=i+1
    
    

0 fold
Epoch 1/100
24463/24463 [==============================] - 21s 850us/step - loss: 4.3401 - acc: 0.2091
Epoch 2/100
24463/24463 [==============================] - 17s 712us/step - loss: 4.3400 - acc: 0.2271
Epoch 3/100
24463/24463 [==============================] - 17s 706us/step - loss: 4.3399 - acc: 0.2251
Epoch 4/100
24463/24463 [==============================] - 18s 738us/step - loss: 4.3399 - acc: 0.2577
Epoch 5/100
24463/24463 [==============================] - 19s 781us/step - loss: 4.3399 - acc: 0.2299
Epoch 6/100
24463/24463 [==============================] - 20s 799us/step - loss: 4.3398 - acc: 0.2440
Epoch 7/100
24463/24463 [==============================] - 19s 760us/step - loss: 4.3398 - acc: 0.2261
Epoch 8/100
24463/24463 [==============================] - 19s 779us/step - loss: 4.3396 - acc: 0.2423
Epoch 9/100
24463/24463 [==============================] - 19s 776us/step - loss: 4.3390 - acc: 0.2739
Epoch 10/100
24463/24463 [==============================] - 18s 74

ValueError: Found input variables with inconsistent numbers of samples: [5, 7645]

# 新闻

In [0]:
df1 = pd.read_csv(r'./TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./20190506_STOCK.csv')
df_test = pd.read_csv(r'./20190514_STOCK.csv')
df1 = pd.concat([df1,df_add,df_test])

In [0]:
df2=df1[['name','ts_code']]
df2 = df2.drop_duplicates(keep='first')


,0,1152,2304,3456,3656,4608,5760,6912,8064,9216,10368,11520,12672,13824,14976,16128,17280,17480,18432,19584,20736,21888,23040,24192,25344,26496,27648,28800,29952,31104,32256,33408,34560,35712,36864,38016
name,农林牧渔,采掘,化工,黑色金属,钢铁,有色金属,电子,家用电器,食品饮料,纺织服装,轻工制造,医药生物,公用事业,交通运输,房地产,商业贸易,餐饮旅游,休闲服务,综合,申万制造,申万消费,申万投资,申万服务,申万300指数,建筑材料,建筑装饰,电气设备,国防军工,计算机,传媒,通信,银行,非银金融,汽车,机械设备,交银装备
ts_code,801010,801020,801030,801040,801040,801050,801080,801110,801120,801130,801140,801150,801160,801170,801180,801200,801210,801210,801230,801250,801260,801270,801280,801300,801710,801720,801730,801740,801750,801760,801770,801780,801790,801880,801890,802600


In [0]:
df_news1 = pd.read_csv(r'./TRAINSET_NEWS.csv')
df_news2= pd.read_csv(r'./20190506_NEWS.csv')
df_news3= pd.read_csv(r'./20190514_NEWS.csv')
df_news = pd.concat([df_news1,df_news2,df_news3])
df_news.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27077 entries, 0 to 112
Data columns (total 4 columns):
id         27077 non-null object
date       27077 non-null int64
title      27048 non-null object
content    26915 non-null object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


# Tensorflow

In [0]:
import tensorflow as tf
time_step=10      #时间步
rnn_unit=25       #hidden layer units
batch_size=80     #每一批次训练多少个样例
input_size=3      #输入层维度
output_size=1     #输出层维度
lr=0.0006 


In [0]:
# 训练集
def get_train_data(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    batch_index=[]
    data_train=dataset[train_begin:train_end]
    normalized_train_data=(data_train-np.mean(data_train,axis=0))/np.std(data_train,axis=0)  #标准化
    train_x,train_y=[],[]   #训练集x和y初定义
    for i in range(len(normalized_train_data)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
            x=normalized_train_data[i:i+time_step,:8]
            y=normalized_train_data[i:i+time_step,7,np.newaxis]
            train_x.append(x.tolist())
            train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    return batch_index,train_x,train_y
 


In [0]:
# 测试集
def get_test_data(time_step=10,test_begin=30000):
    data_test=dataset[test_begin:]
    mean=np.mean(data_test,axis=0)
    std=np.std(data_test,axis=0)
    normalized_test_data=(data_test-mean)/std  #标准化
    size=(len(normalized_test_data)+time_step-1)//time_step  #有size个sample 
    test_x,test_y=[],[]  
    for i in range(size-1):
        x=normalized_test_data[i*time_step:(i+1)*time_step,:8]
        y=normalized_test_data[i*time_step:(i+1)*time_step,7]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i+1)*time_step:,:3]).tolist())
    test_y.extend((normalized_test_data[(i+1)*time_step:,2]).tolist())
    return mean,std,test_x,test_y

In [0]:
# 模型搭建
weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
}
def lstm(X):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']  
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states

In [0]:
def train_lstm(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step,train_begin,train_end)
    pred=lstm(X)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
    module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        #sess.run(tf.global_variables_initializer())
        saver.restore(sess, module_file)
        #重复训练2000次
        for i in range(2000):
            for step in range(len(batch_index)-1):
                loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]]})
            print(i,loss_)
            if i % 200==0:
                print("保存模型：",saver.save(sess,'stock{i}.model',global_step=i))